In [ ]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------

from google.colab import files
# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, MaxAbsScaler

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

In [ ]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# -------------------------------------

In [ ]:
uploaded = files.upload()

train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

Saving annotated_corpus_for_train.csv to annotated_corpus_for_train (1).csv
Saving corpus_for_test.csv to corpus_for_test (1).csv


In [ ]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [ ]:
df_test.head(8)
df_test.shape

,test_example_id,story,predicted_gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,f
5,5,בפעם האחרונה שהייתי מחוץ לארץ ישראל הייתי באפר...,m
6,6,בשנת 2018 קיבלתי החלטה שאני מתחיל ללמוד לתואר ...,m
7,7,"בנובמבר האחרון הייתי עם חברים בטיול ים אל ים, ...",m


(323, 3)

# Preparing the data for a supervised learning


---




In [ ]:
X_train = df_train["story"]
y_train = df_train["gender"]

# A raw code with 1 model and no Hyperparameters, just for testing

---



In [ ]:
pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=5)),
        ('norm', preprocessing.Normalizer(norm='l1')),
        ('model', Perceptron())
    ])
 
cross_val = cross_val_score(pipeline, X_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10).mean()
print(cross_val)

0.5856652611283314


# Creating models list

*I'll use this list to perform grid search later on and evaluate these classifiers.*

---



In [ ]:
models = [
    ('Perceptron', Perceptron()),
    ('LinearSVC', LinearSVC()),
    ('DecisionTreeClassifier', DecisionTreeClassifier()),
    ('MultinomialNB', MultinomialNB()),
    ('SGDClassifier', SGDClassifier()),
    ('KNeighborsClassifier', KNeighborsClassifier())
]

# Making `set_parameters` function:

Sets the grid search parameters for each classifier based on its name. For now, it will just take some Hyperparameters for the models.

---




In [ ]:
def set_parameters():
    if name == 'LinearSVC':
        parameters = {'model__C': [0.1, 1, 10], 'model__penalty': ['l1', 'l2']}
    elif name == 'Perceptron':
        parameters = {'model__alpha': [0.0001, 0.001, 0.01], 'model__penalty': ['l1', 'l2']}
    elif name == 'DecisionTreeClassifier':
        parameters = {'model__max_depth': [5, 10, 15], 'model__min_samples_leaf': [5, 10, 15]}
    elif name == 'MultinomialNB':
        parameters = {'model__alpha': [0.01, 0.1, 1]}
    elif name == 'SGDClassifier':
        parameters = {'model__alpha': [1e-3, 1e-4], 'model__penalty': ['l1', 'l2']}
    elif name == 'KNeighborsClassifier':
        parameters = {'model__n_neighbors': [3, 5, 7]}
    
    return parameters

In this part, the code iterats over each model and creats a pipeline with the specified steps, sets the parameters for grid search using the `set_parameters()` function. Then it will perform grid search with cross-validation using `GridSearchCV` for each model, searching over the specified parameter grid. It will then print the best score, and the best estimator.

**best_estimator_**: 

*Estimator that was chosen by the search, i.e. estimator which gave highest score (or smallest loss if specified) on the left out data.*

**best_score_**:

Mean cross-validated score of the best_estimator

[sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

---




In [ ]:
for name, model in models:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('norm', preprocessing.Normalizer(norm='l1')),
        ('model', model)
    ])

    parameters = set_parameters()

    grid = GridSearchCV(pipeline, parameters, scoring=make_scorer(f1_score, average='macro'), cv=10)
    grid.fit(X_train, y_train)

    print("Best score is:", round(grid.best_score_, 3))
    print("Best estimator:", grid.best_estimator_)



GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer(norm='l1')),
                                       ('model', Perceptron())]),
             param_grid={'model__alpha': [0.0001, 0.001, 0.01],
                         'model__penalty': ['l1', 'l2']},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.534
Beste estimator: Pipeline(steps=[('tfidf', TfidfVectorizer()), ('norm', Normalizer(norm='l1')),
                ('model', Perceptron(penalty='l2'))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer(norm='l1')),
                                       ('model', LinearSVC())]),
             param_grid={'model__C': [0.1, 1, 10],
                         'model__penalty': ['l1', 'l2']},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.433
Beste estimator: Pipeline(steps=[('tfidf', TfidfVectorizer()), ('norm', Normalizer(norm='l1')),
                ('model', LinearSVC(C=0.1))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer(norm='l1')),
                                       ('model', DecisionTreeClassifier())]),
             param_grid={'model__max_depth': [5, 10, 15],
                         'model__min_samples_leaf': [5, 10, 15]},
             scoring=make_scorer(f1_score, average=macro))

...

Best score is: 0.433
Beste estimator: Pipeline(steps=[('tfidf', TfidfVectorizer()), ('norm', Normalizer(norm='l1')),
                ('model', MultinomialNB(alpha=0.01))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer(norm='l1')),
                                       ('model', SGDClassifier())]),
             param_grid={'model__alpha': [0.001, 0.0001],
                         'model__penalty': ['l1', 'l2']},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.433
Beste estimator: Pipeline(steps=[('tfidf', TfidfVectorizer()), ('norm', Normalizer(norm='l1')),
                ('model', SGDClassifier(alpha=0.001, penalty='l1'))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer(norm='l1')),
                                       ('model', KNeighborsClassifier())]),
             param_grid={'model__n_neighbors': [3, 5, 7]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.455
Beste estimator: Pipeline(steps=[('tfidf', TfidfVectorizer()), ('norm', Normalizer(norm='l1')),
                ('model', KNeighborsClassifier(n_neighbors=3))])


# Not good results at all!

I figured out I should also check some other parameters for the `TfidfVectorizer()` and the `preprocessing.Normalizer()`.

**So now I will try to change the `set_parameters()` function and the pipeline accordingly**

I will try multiple combinations - it`s probabaly gonna take some time.

---




In [ ]:
def set_parameters():
    if name == 'LinearSVC':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df' : [2, 4, 5], 'tfidf__max_df': [0.8, 0.9],
                      'model__C': [0.1, 1, 10], 'model__penalty': ['l1', 'l2']}

    elif name == 'Perceptron':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df' : [2, 4, 5], 'tfidf__max_df': [0.8, 0.9], 'tfidf__sublinear_tf':[True, False],
                    'model__alpha': [0.000001, 0.00001, 0.0001], 'model__penalty': ['l2', 'l1']}

    elif name == 'DecisionTreeClassifier':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df' : [2, 4, 5], 'tfidf__max_df': [0.8, 0.9],
                      'model__max_depth': [5, 10, 15], 'model__min_samples_leaf': [5, 10, 15]}

    elif name == 'MultinomialNB':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df' : [2, 4, 5], 'tfidf__max_df': [0.8, 0.9], 
                      'model__alpha': [0.01, 0.1, 1]}

    elif name == 'SGDClassifier':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df' : [2, 4, 5], 'tfidf__max_df': [0.8, 0.9],
                      'model__alpha': [1e-3, 1e-4], 'model__penalty': ['l1', 'l2']}

    elif name == 'KNeighborsClassifier':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df' : [2, 4, 5], 'tfidf__max_df': [0.8, 0.9],
                      'model__n_neighbors': [3, 5, 7], 'model__metric': ['euclidean', 'manhattan', 'chebyshev'], 'model__leaf_size': [10, 20, 30]}

    return parameters


In [ ]:
for name, model in models:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('norm', preprocessing.Normalizer(norm = 'l2')),
        ('model', model)
    ])
  

    parameters = set_parameters()

    grid = GridSearchCV(pipeline, parameters, scoring=make_scorer(f1_score, average='macro'), cv=10)
    grid.fit(X_train, y_train)


    print("Best score is:", round(grid.best_score_, 3))
    print("Best estimator:", grid.best_estimator_)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', Perceptron())]),
             param_grid={'model__alpha': [1e-06, 1e-05, 0.0001],
                         'model__penalty': ['l2', 'l1'],
                         'tfidf__max_df': [0.8, 0.9],
                         'tfidf__min_df': [2, 4, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)],
                         'tfidf__sublinear_tf': [True, False]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.726
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('norm', Normalizer()),
                ('model', Perceptron(alpha=1e-06, penalty='l1'))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', LinearSVC())]),
             param_grid={'model__C': [0.1, 1, 10],
                         'model__penalty': ['l1', 'l2'],
                         'tfidf__max_df': [0.8, 0.9],
                         'tfidf__min_df': [2, 4, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.687
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1, 2))),
                ('norm', Normalizer()), ('model', LinearSVC(C=10))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', DecisionTreeClassifier())]),
             param_grid={'model__max_depth': [5, 10, 15],
                         'model__min_samples_leaf': [5, 10, 15],
                         'tfidf__max_df': [0.8, 0.9],
                         'tfidf__min_df': [2, 4, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

...

Best score is: 0.526
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=5, ngram_range=(1, 2))),
                ('norm', Normalizer()), ('model', MultinomialNB(alpha=0.01))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', SGDClassifier())]),
             param_grid={'model__alpha': [0.001, 0.0001],
                         'model__penalty': ['l1', 'l2'],
                         'tfidf__max_df': [0.8, 0.9],
                         'tfidf__min_df': [2, 4, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.728
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=4, ngram_range=(1, 2))),
                ('norm', Normalizer()),
                ('model', SGDClassifier(penalty='l1'))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', KNeighborsClassifier())]),
             param_grid={'model__leaf_size': [10, 20, 30],
                         'model__metric': ['euclidean', 'manhattan',
                                           'chebyshev'],
                         'model__n_neighbors': [3, 5, 7],
                         'tfidf__max_df': [0.8, 0.9],
                         'tfidf__min_df': [2, 4, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.613
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2))),
                ('norm', Normalizer()),
                ('model',
                 KNeighborsClassifier(leaf_size=10, metric='euclidean'))])


## Significant Improvements Achieved in Model Performance!
After investing some time and effort, it is clear that the `SGDClassifier()` and `Perceptron()` models have shown the most promising results. These models outperformed the others and demonstrated superior performance.

Now that we have identified these two models as the top performers, we can focus our attention on further tuning them to try achieve even better results. 

---



In [ ]:
chosen_models = [
    ('Perceptron', Perceptron()),
    ('SGDClassifier', SGDClassifier()),
]

In [ ]:
def set_parameters_for_the_chosen_models():
    if name == 'Perceptron':
         parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)],'tfidf__min_df': [2, 3, 6], 'tfidf__max_df': [0.8, 0.6, 0.7], 'tfidf__sublinear_tf': [True, False],
                      'norm__norm': ['l1', 'l2'],
                      'model__alpha': [0.000001, 0.001, 0.01], 'model__penalty': ['l2', 'l1'], 'model__fit_intercept': [True, False], 'model__shuffle': [True, False]}
    
    elif name == 'SGDClassifier':
        parameters = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__min_df': [4, 3, 6], 'tfidf__max_df': [0.8, 0.6. 0.7], 
                      'norm__norm': ['l1', 'l2'],
                      'model__alpha': [1e-3, 1e-4], 'model__penalty': ['l1', 'l2'], 'model__fit_intercept': [True, False], 'model__loss': ['hinge', 'log', 'modified_huber'],
                      'model__epsilon': [0.1, 0.01, 0.001]}

    return parameters


In [ ]:
best_score_overall = 0.0
best_model_overall = None

for name, model in chosen_models:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('norm', preprocessing.Normalizer()),
        ('model', model)
    ])
  

    parameters = set_parameters_for_the_chosen_models()

    grid = GridSearchCV(pipeline, parameters, scoring=make_scorer(f1_score, average='macro'), cv=10)
    grid.fit(X_train, y_train)

    if best_score_overall < grid.best_score_:
        best_score_overall = grid.best_score_
        best_model_overall = grid.best_estimator_

    print("Best score is:", round(grid.best_score_, 3))
    print("Best estimator:", grid.best_estimator_)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', Perceptron())]),
             param_grid={'model__alpha': [1e-06, 0.001, 0.01],
                         'model__fit_intercept': [True, False],
                         'model__penalty': ['l2', 'l1'],
                         'model__shuffle': [True, False],
                         'norm__norm': ['l1', 'l2'],
                         'tfidf__max_df': [0.8, 0.6],
                         'tfidf__min_df': [2, 3, 6],
                         'tfidf__ngram_range': [(1, 1), (1, 2)],
                         'tfidf__sublinear_tf': [True, False]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.726
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('norm', Normalizer()),
                ('model', Perceptron(alpha=1e-06, penalty='l1'))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', SGDClassifier())]),
             param_grid={'model__alpha': [0.001, 0.0001],
                         'model__epsilon': [0.1, 0.01, 0.001],
                         'model__fit_intercept': [True, False],
                         'model__loss': ['hinge', 'log', 'modified_huber'],
                         'model__penalty': ['l1', 'l2'],
                         'norm__norm': ['l1', 'l2'],
                         'tfidf__max_df': [0.8, 0.6],
                         'tfidf__min_df': [4, 3, 6],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.715
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=4, ngram_range=(1, 2))),
                ('norm', Normalizer()),
                ('model', SGDClassifier(epsilon=0.001, penalty='l1'))])


In [ ]:
print(best_model_overall)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('norm', Normalizer()),
                ('model', Perceptron(alpha=1e-06, penalty='l1'))])


# Ultimately, I was unable to further enhance the performance of the two models.
Consequently, I will proceed with utilizing the model that yielded the most favorable outcomes to make predictions for the initial five stories as well as the final five stories. 

I will set the parameters which have shown the best results.

---



In [61]:
best_score_overall = 0.0
best_model = None


best_chosen_models = [
    ('Perceptron', Perceptron()),
    ('SGDClassifier', SGDClassifier()),
]

def set_parameters_for_the_best_chosen_models():
    if name == 'Perceptron':
         parameters = {'tfidf__ngram_range': [(1, 2)],'tfidf__min_df': [2], 'tfidf__max_df': [0.8], 'tfidf__sublinear_tf': [True],
                      'model__alpha': [0.000001], 'model__penalty': ['l1']}
    elif name == 'SGDClassifier':
        parameters = {'tfidf__ngram_range': [(1, 2)], 'tfidf__min_df': [4], 'tfidf__max_df': [0.8], 
                      'model__alpha': [1e-3, 1e-4], 'model__penalty': ['l1']}

    return parameters


for name, model in best_chosen_models:
    pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(norm = 'l2')),
    ('norm', preprocessing.Normalizer()),
    ('model', model)
    ])


    parameters = set_parameters_for_the_best_chosen_models()

    grid = GridSearchCV(pipeline, parameters, scoring=make_scorer(f1_score, average='macro'), cv=10)
    grid.fit(X_train, y_train)

    if best_score_overall < grid.best_score_:
        best_score_overall = grid.best_score_
        best_model = grid.best_estimator_


    print("Best score is:", round(grid.best_score_, 3))
    print("Best estimator:", grid.best_estimator_)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', Perceptron())]),
             param_grid={'model__alpha': [1e-06], 'model__penalty': ['l1'],
                         'tfidf__max_df': [0.8], 'tfidf__min_df': [2],
                         'tfidf__ngram_range': [(1, 2)],
                         'tfidf__sublinear_tf': [True]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.726
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('norm', Normalizer()),
                ('model', Perceptron(alpha=1e-06, penalty='l1'))])


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm', Normalizer()),
                                       ('model', SGDClassifier())]),
             param_grid={'model__alpha': [0.001, 0.0001],
                         'model__penalty': ['l1'], 'tfidf__max_df': [0.8],
                         'tfidf__min_df': [4], 'tfidf__ngram_range': [(1, 2)]},
             scoring=make_scorer(f1_score, average=macro))

Best score is: 0.705
Best estimator: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=4, ngram_range=(1, 2))),
                ('norm', Normalizer()),
                ('model', SGDClassifier(penalty='l1'))])


## The final model:

The `Perceptron()` model will be chosen over the `SGDCclassifier()` model due to its perceived higher reliability. The `SGDCclassifier()` model exhibits inconsistent scores across multiple runs, which poses a significant concern. In contrast, the `Perceptron()` model consistently maintains its performance throughout various iterations.


In [ ]:
def make_pipeline_for_chosen_model():
    pipeline_per = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2), sublinear_tf=True)),
    ('norm', preprocessing.Normalizer(norm = 'l2')),
    ('model', Perceptron(alpha=1e-06, penalty='l1'))
    ])

    return pipeline_per

In [78]:
model = make_pipeline_for_chosen_model()

# Fit the TF-IDF vectorizer on training data
X_train = df_train["story"]
model.fit(X_train, y_train)

# Use the trained model to predict labels for the test data
X_test = df_test["story"]
y_pred = model.predict(X_test)
print(model)

# Combine the predicted labels with the stories in the test dataframe
df_test['predicted_gender'] = y_pred

# Print the top 5 and bottom 5 stories
print('Top 5 Stories:')
df_test.head(10)
print('\nBottom 5 Stories:')
df_test.tail(10)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('norm', Normalizer()),
                ('model', Perceptron(alpha=1e-06, penalty='l1'))])

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2),
                                 sublinear_tf=True)),
                ('norm', Normalizer()),
                ('model', Perceptron(alpha=1e-06, penalty='l1'))])
Top 5 Stories:


,test_example_id,story,predicted_gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
5,5,בפעם האחרונה שהייתי מחוץ לארץ ישראל הייתי באפר...,m
6,6,בשנת 2018 קיבלתי החלטה שאני מתחיל ללמוד לתואר ...,m
7,7,"בנובמבר האחרון הייתי עם חברים בטיול ים אל ים, ...",m
8,8,"לפני מספר חודשים, ביום שמש בהיר, קמתי בבוקר למ...",m
9,9,"אני לא בן אדם שנוטה לשתף בחייו האישיים, אבל או...",m



Bottom 5 Stories:


,test_example_id,story,predicted_gender
313,313,"שלום, לפני כמה חודשים אני ועוד שני חברים מהילד...",m
314,314,"השנה לא הייתה לנו חופשה אמיתית, חווינו סגרים, ...",f
315,315,"לפני כמה חודשים, טסתי לאוקראינה ובאותו ביקור ה...",m
316,316,בנסיעה האחרונה שלי לחול במהלך הקורונה החלטתי ש...,f
317,317,לפני כמה שנים החלטתי ללכת לקורס גלישה ביחד עם ...,f
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m
